# End-to-end Twitter SPI

In [2]:
from pysentimiento.preprocessing import preprocess_tweet
from transformers                import AutoTokenizer, AutoModelForSequenceClassification
from pysentimiento               import create_analyzer

import pandas as    pd
import numpy as     np
import torch

In [3]:
def rescale_probs(proba_dict):
    keys, values = list(proba_dict.keys()), list(proba_dict.values())
    pred_key = keys[np.argmax(values)]
    pred_value = 0
    if pred_key == "NEG":
        pred_value = 1-np.max(values)
    elif pred_key == "POS":
        pred_value = np.max(values)
    else:
        neg_value = values[0]
        pos_value = values[-1]

        add_val = pos_value if pos_value > neg_value else -neg_value

        pred_value = 0.5 + (1-np.max(values))*add_val/2
    return pred_value

In [9]:
# tokenizer = AutoTokenizer.from_pretrained("papluca/xlm-roberta-base-language-detection")
# language_detector = AutoModelForSequenceClassification.from_pretrained("papluca/xlm-roberta-base-language-detection")
# analyzer_en = create_analyzer(task="sentiment", lang="en")
# analyzer_es = create_analyzer(task="sentiment", lang="es")


def preprocess_tweet_adv(string_, proc_func):
    substrs_to_remove = ["cara emoji", "emoji", "   ", "\n"]
    procs_str = proc_func(string_)
    for substr in substrs_to_remove:
        procs_str = procs_str.replace(substr, "")
    procs_str = procs_str.replace("Jjaja", "Jajaja")

    return procs_str


def get_post_sentiment(comments, lang_detector, en_analyzer, sp_analyzer, tokenizer):
    preprocessed_comments = [preprocess_tweet_adv(comment) for comment in comments]
    languages_detected = []
    for comment in preprocessed_comments:
        inputs = tokenizer(comment, return_tensors="pt")
        with torch.no_grad():
            logits = lang_detector(**inputs).logits
        predicted_class_id = logits.argmax().item()
        languages_detected.append(lang_detector.config.id2label[predicted_class_id])

    sentiment_probas = []
    for ix in range(len(preprocessed_comments)):
        if languages_detected[ix] == "en":
            estimation = en_analyzer.predict(preprocessed_comments[ix]).probas
        elif languages_detected[ix] == "es" or languages_detected[ix] == "pt":
            estimation = sp_analyzer.predict(preprocessed_comments[ix]).probas
        else:
            estimation = {'NEG': 0, 'NEU': 1, 'POS': 0}
        sentiment_probas.append(estimation)

    sentiment = [rescale_probs(probs) for probs in sentiment_probas]
    df = pd.DataFrame({"Comment": comments, "Language": languages_detected, "Sentiment": sentiment})

    return df, df["Sentiment"].mean()


In [5]:
data_user = {'user_name': 'JFCadavid',
 'n_retweets': 4,
 'n_tweets': 6,
 'n_retweets_to_user': [0, 1, 0, 18, 1, 2],
 'n_favorites_to_user': [0, 10, 6, 801, 282, 6],
 'tweets_text': ['@RondoSendo @JuniorClubSA Jajajaja',
  '¿Quieres ganar entrada doble para el partido entre DIM 🆚 Águilas, y camisa del Medellin ? \n\nPasos: \nHaz un video co… https://t.co/KWizbpyJd1',
  '¡EN VIVO ANALIZANDO LA SALIDA DE JULIO COMESAÑA!\n\n¿La culpa del momento del Junior era de él o de alguien más?\n\n¿El… https://t.co/KafYtJL2f3',
  'Aterrizando en Doha y me entero de la salida de Julio Comesaña… ¿qué pasó?',
  '@josechazyvip Ninguna, es más fácil que ud me deje de seguir… La gente es boba sufriendo gratis',
  '¿Quieren ganar boletas para ver \n👉 Medellín 🆚 Águilas Doradas\n👉 Pereira 🆚 Millonarios ?'],
 'tweets_replies': [[],
  ['@JFCadavid @PapasMargarita Para que quiero tus entradas Juan bobipe Guevavid',
   '@JFCadavid @PapasMargarita Pilas antes que se implemente la reforma tributaria',
   '@JFCadavid @PapasMargarita Está reñido mucha gente participando 😂😂😂',
   '@JFCadavid @PapasMargarita @JFCadavid @PapasMargarita https://t.co/Cnu7BxvDCv',
   '@JFCadavid @PapasMargarita Juan yo participe ayer en instagram. Es lo mismo ?'],
  ['@JFCadavid Cállate Juan bobipe.',
   '@JFCadavid Llegó al junior para cagarse con los azules, técnico mediocre'],
  ['@JFCadavid 🤣',
   '@JFCadavid Que ganas de figurar que está en Catar 🥱',
   '@JFCadavid Lo necesitan para después de la pretemporada. Cuando vuelvan a sacar el nuevo técnico. @JFCadavid Junior y @nacionaloficial han demostrado que la grandeza no se puede medir con estas directivas.',
   '@JFCadavid Jajjajajjajjaaja duró más la selección Colombia en Qatar... momento 😡',
   '@JFCadavid Lo que tenía que pasar amigo',
   '@JFCadavid Salió llorando porque millonarios ganó la copa y lo tiene de hijo',
   '@JFCadavid Crónica de un fracaso anunciado.  Pero ya están contratando a Julio XI',
   '@JFCadavid Comesaña se fue porque están en conversaciones con Julio Avelino y el zurdo López',
   '@JFCadavid @ORTEMAN10 😂😂😂😂😂',
   '@JFCadavid Pues lo de siempre, no hay n proyecto deportivo serio, siempre se toman decisiones basadas en caprichos y pataletas. La pregunta es para qué lo trajeron?'],
  ['@JFCadavid @josechazyvip Q buena respuesta',
   '@JFCadavid @josechazyvip Como periodista no le queda bien apuntando a la gente como.boba. debería de rectificar, ya que en otras opiniones no se le puede decir nada a usted porque ya sabemos que responderá con agravios.pilas con eso.',
   '@JFCadavid @josechazyvip Jajajaja excelente respuesta!!!',
   '@JFCadavid @josechazyvip Jajajaa peinadota',
   '@JFCadavid @josechazyvip Pues yo no lo sigo, y me aparecen tus comentarios',
   '@JFCadavid @josechazyvip dele picos a la calva de su marido, y se quedan trabajando allá, pero con el *',
   '@JFCadavid @josechazyvip Que peinada linda',
   '@JFCadavid @josechazyvip Si lo bloqueas es mejor',
   '@JFCadavid @josechazyvip Dile que se vista Jjaajajajajajaajajajaajajajaja',
   '@JFCadavid @josechazyvip El Junior ahora compite con el SENA, para conocer quién saca más técnicos... Está buena la contienda... Saludos !! Que disfrute bastante!!'],
  ['@JFCadavid no aguanta el campeón ya se sabe quien será o no Millos?',
   '@JFCadavid Naaaa 🥱🥱🥱🥱🥱🥱',
   '@JFCadavid Juan Felipe bolillo ganó una liga con nacional \nEl entrenador de nacional vs gremio era Juan José pelaes',
   '@JFCadavid Como sería? Pero no papas, no consumo paquetes Juan! 😅',
   '@JFCadavid Medellín 💙❤️']],
 'date': '18/11/2022'}

In [6]:
comments_a = data_user['tweets_replies'][-1]
comments_a

['@JFCadavid no aguanta el campeón ya se sabe quien será o no Millos?',
 '@JFCadavid Naaaa 🥱🥱🥱🥱🥱🥱',
 '@JFCadavid Juan Felipe bolillo ganó una liga con nacional \nEl entrenador de nacional vs gremio era Juan José pelaes',
 '@JFCadavid Como sería? Pero no papas, no consumo paquetes Juan! 😅',
 '@JFCadavid Medellín 💙❤️']

In [53]:
df, sentiment_val = get_post_sentiment(comments_a, language_detector, analyzer_en, analyzer_es, tokenizer)

In [54]:
comments_a

['@JFCadavid @josechazyvip Q buena respuesta',
 '@JFCadavid @josechazyvip Como periodista no le queda bien apuntando a la gente como.boba. debería de rectificar, ya que en otras opiniones no se le puede decir nada a usted porque ya sabemos que responderá con agravios.pilas con eso.',
 '@JFCadavid @josechazyvip Jajajaja excelente respuesta!!!',
 '@JFCadavid @josechazyvip Jajajaa peinadota',
 '@JFCadavid @josechazyvip Pues yo no lo sigo, y me aparecen tus comentarios',
 '@JFCadavid @josechazyvip dele picos a la calva de su marido, y se quedan trabajando allá, pero con el *',
 '@JFCadavid @josechazyvip Que peinada linda',
 '@JFCadavid @josechazyvip Si lo bloqueas es mejor',
 '@JFCadavid @josechazyvip Dile que se vista Jjaajajajajajaajajajaajajajaja',
 '@JFCadavid @josechazyvip El Junior ahora compite con el SENA, para conocer quién saca más técnicos... Está buena la contienda... Saludos !! Que disfrute bastante!!']

In [55]:
df

,Comment,Language,Sentiment
0,@JFCadavid @josechazyvip Q buena respuesta,es,0.987111
1,@JFCadavid @josechazyvip Como periodista no le...,es,0.001619
2,@JFCadavid @josechazyvip Jajajaja excelente re...,es,0.994382
3,@JFCadavid @josechazyvip Jajajaa peinadota,pt,0.920389
4,"@JFCadavid @josechazyvip Pues yo no lo sigo, y...",es,0.012591
5,@JFCadavid @josechazyvip dele picos a la calva...,es,0.449074
6,@JFCadavid @josechazyvip Que peinada linda,pt,0.988774
7,@JFCadavid @josechazyvip Si lo bloqueas es mejor,es,0.499490
8,@JFCadavid @josechazyvip Dile que se vista Jja...,es,0.501064
9,@JFCadavid @josechazyvip El Junior ahora compi...,es,0.935286


In [11]:
preprocessed_comments = [preprocess_tweet_adv(comment, preprocess_tweet) for comment in comments_a]
preprocessed_comments

['@usuario no aguanta el campeón ya se sabe quien será o no Millos?',
 '@usuario Naaacara de bostezo cara de bostezo cara de bostezo ',
 '@usuario Juan Felipe bolillo ganó una liga con nacional El entrenador de nacional vs gremio era Juan José pelaes',
 '@usuario Como sería? Pero no papas, no consumo paquetes Juan!cara sonriendo con sudor frío ',
 '@usuario Medellíncorazón azul ']